参考书：《密码学C/C++语言实现》

In [1]:
from cryptography.fernet import Fernet
key = Fernet.generate_key()
f = Fernet(key)
token = f.encrypt(b"A really secret message. Not for prying eyes.")
print(token)
print(f.decrypt(token))

b'gAAAAABl78P_R7jfwO7V3YsHKvrA-aS_GSo16QJeKpgKI6mX0bJ5gkIi6_tC6Y66S0FG-37DoLjeYe9V6nyuCMAKcu9m-ZSmLAQLXKlXv_gkc-kG7E5YBxfy_cqkOgc8VasanS_pD5tU'
b'A really secret message. Not for prying eyes.'


In [12]:
from cryptography.fernet import Fernet
key = Fernet.generate_key()
f = Fernet(key)
# b表示把文本编码成字符的形式，只是对于ASCII码的，不能编码中文
plaintext = "密码学实验"
# 对于中文字符，先encode一下
plaintext = plaintext.encode()
# token代表密文，对token解密得到的是解密得到的encode后的，如果要得到中文，还需要decode一下
token = f.encrypt(plaintext)
print(token)
print(f.decrypt(token))
print(f.decrypt(token).decode())

b'gAAAAABl5wXqx8cLZ8cN1pyOByOfYygDQKHIqmAtcq15nYAlAx8jQT3HBFf-30IYiY7pvm_HEEwMTkNvu6EdiFkPGRScaC92Ag=='
b'\xe5\xaf\x86\xe7\xa0\x81\xe5\xad\xa6\xe5\xae\x9e\xe9\xaa\x8c'
密码学实验


In [11]:
# 调用AES算法，如何去加密（一段英文）
import os
from cryptography.hazmat.primitives.ciphers import Cipher,algorithms,modes
# .ciphers表示调用primitive里面的算法
# cipher表示实现加密解密功能的，不能只有算法，还有工作模式等等，攻击的是cipher
# algorithm表示给定分组大小、组件等等
# primitive表示密码学中大块的研究方向，比如对称密码、分组密码、认证加密等等，但是AES算法之类的是不能成为primitive
plaintext = b"this is a test message, hahahahahaha~"
# 随机生成一个密钥，生成一个32字节/256比特（密钥长度，可以是128、196、256三种），分组长度固定128比特
# 如果一个文本不满128比特，就需要用一些工作模式，GCM是对于AES应用最多的一种工作模式，是把AES当作流密码用了
# 6种工作模式种CTR和GCM可以将分组密码转换成流密码使用
# 3GPP推出的3个标准：ZUC、Schnorr-3G:基于字生成密钥流、AES-GCM
# 流密码：给密钥流生成器输入一个种子密钥、IV经过初始化生成密钥流，长度任意，通过算法反复执行，将密钥流和明文XOR逐比特加密
# CTR模式（速度快、加解密方便，类似于流密码）：输入一个密钥，用分组密码加密计数器的值，得到输出，再和明文XOR加密即可得到了密文。由于明文和分组密码的输出异或，所以需要吧明文按照分组密码的大小进行分块
# 有多少块就是用多少个计数器
# 计数器：需要生成，随机生成一个Nonce或者IV，然后做一个处理，作为第一个计数器的值，然后分成两块，前面一块称为随机值，保持不动，后面一块才称为计数器，是要递增的
# 划分原则：后面一块要每次加1，作为counter的这一部分要保证明文划分的分块都要遍历完
# 填充：有很多填充方法：PKCS5等、即使最后一个块是完整的也可以填充，也可以不填充，不填充就仍然是逐比特异或，最后剩下多少比特，就用多少比特就好了

# 使用32bytes的key，即使用算法AES-256-CTR
key = os.urandom(32)
# iv长度要和分组长度一样,AES算法的分块大小固定128，即16字节
# CTR划分为堆积部分和递增部分，递增部分值增加，生成计数器序列
iv = os.urandom(16)

#1.发送方加密数据
# 构建AES-256-CTR的cipher，然后加密数据得到密文
cipher = Cipher(algorithms.AES(key),modes.CTR(iv))
# 实例化一个加密函数
encryptor = cipher.encryptor()
# 放到update就是明文，finalize是检验有没有认证，任何都有这两部分
ciphertext = encryptor.update(plaintext)+encryptor.finalize()
print(ciphertext)
# 2.发送方将iv+ciphertext发送给接收方
# 3.接收方解密
# # 接收方使用自己的key+接收到的iv，构建cipher，然后解密出原始数据
cipher = Cipher(algorithms.AES(key),modes.CTR(iv))
decryptor = cipher.decryptor()
plaintext = decryptor.update(ciphertext)+decryptor.finalize()
print(plaintext)

# 如果加密的是中文？加密模式如果不用CTR


b'\xf0\xf0:\x99O\xe0e\x1d\x07\xbe&\x13%\x97\xae\x84%\xf9\xc6\x8c\x18pf\xccT\xb4\xa1\xe5Z\x19\x88\xa3\x17\x1b[`*'
b'this is a test message, hahahahahaha~'


CBC模式：搜一下cryptography的文档，看填充如何解决，如何调用这个库中的填充
需要填充，明文长度与密文异或的时候，位数要一致
GCM模式：数据
CTR可以不填充
SM4算法：


区分：差分路径trail与差分特征character

In [19]:
# 调用AES算法，如何去加密（一段英文）
import os
from cryptography.hazmat.primitives.ciphers import Cipher,algorithms,modes
plaintext = b"this is a test message, hahahahahaha~"
key = os.urandom(32)
iv = os.urandom(16)

#1.发送方加密数据
# 构建AES-256-CTR的cipher，然后加密数据得到密文
cipher = Cipher(algorithms.AES(key),modes.CTR(iv))
# 实例化一个加密函数
encryptor = cipher.encryptor()
# 放到update就是明文，finalize是检验有没有认证，任何都有这两部分
ciphertext = encryptor.update(plaintext)+encryptor.finalize()
# 2.发送方将iv+ciphertext发送给接收方
# 3.接收方解密
# # 接收方使用自己的key+接收到的iv，构建cipher，然后解密出原始数据
cipher = Cipher(algorithms.AES(key),modes.CTR(iv))
decryptor = cipher.decryptor()
decryptor.update(ciphertext)+decryptor.finalize()

b'this is a test message, hahahahahaha~'

In [10]:
# 调用AES算法，如何去加密（一段英文）
import os
from cryptography.hazmat.primitives.ciphers import Cipher,algorithms,modes
plaintext = "密码学实验"
plaintext = plaintext.encode()
key = os.urandom(32)
iv = os.urandom(16)

#1.发送方加密数据
# 构建AES-256-CTR的cipher，然后加密数据得到密文
cipher = Cipher(algorithms.AES(key),modes.CTR(iv))
# 实例化一个加密算法，生成一个加密器
encryptor = cipher.encryptor()
# 放到update就是明文，finalize是检验有没有认证，任何都有这两部分
ciphertext = encryptor.update(plaintext)+encryptor.finalize()
# 2.发送方将iv+ciphertext发送给接收方
# 3.接收方解密
# # 接收方使用自己的key+接收到的iv，构建cipher，然后解密出原始数据
cipher = Cipher(algorithms.AES(key),modes.CTR(iv))
decryptor = cipher.decryptor()
pt = decryptor.update(ciphertext)+decryptor.finalize()
pt.decode()

'密码学实验'

In [9]:
# CBC工作模式
import os
from cryptography.hazmat.primitives.ciphers import Cipher,algorithms,modes
from cryptography.hazmat.primitives import padding
# 生成填充器，有很多填充方式，主要使用的是PKCS7，唯一的参数是填充好之后的长度
padder = padding.PKCS7(128).padder()
pt = b"a secret message"
# 把明文填充成128
pt = padder.update(pt)
key = os.urandom(32)
iv = os.urandom(16)
cipher = Cipher(algorithms.AES(key),modes.CBC(iv))
encryptor = cipher.encryptor()
ciphertext = encryptor.update(pt)+encryptor.finalize()

cipher = Cipher(algorithms.AES(key),modes.CBC(iv))
decryptor = cipher.decryptor()
decryptor.update(ciphertext)+decryptor.finalize()




b'a secret message'

In [8]:
# SM4，只支持128bit的密钥
# CBC工作模式
import os
from cryptography.hazmat.primitives.ciphers import Cipher,algorithms,modes

plaintext = "密码学实验"
plaintext = plaintext.encode()
# 把明文填充成128
key = os.urandom(16)
iv = os.urandom(16)
cipher = Cipher(algorithms.SM4(key),modes.CTR(iv))
encryptor = cipher.encryptor()
ciphertext = encryptor.update(plaintext)+encryptor.finalize()

cipher = Cipher(algorithms.SM4(key),modes.CTR(iv))
decryptor = cipher.decryptor()
decryptor.update(ciphertext)+decryptor.finalize()

b'\xe5\xaf\x86\xe7\xa0\x81\xe5\xad\xa6\xe5\xae\x9e\xe9\xaa\x8c'

In [7]:
# Hash函数，计算消息的Hash值
# SHA3支持的消息摘要长度更灵活
from cryptography.hazmat.primitives import hashes
digest = hashes.Hash(hashes.SHA3_512())
message = b"this is a message"

digest.update(message)
# 和上面分组密码不同，不是放在一起的
# 对于hash函数，都有finalize函数，但对于对称密码算法，有的有，有的没有
# 对于hash函数一定有finalize
# 表示成了字节的形式
digest.finalize()

b'\xf2\x9d\xdb\xf1\xd7\x17\xa7K_\xfe\n\xf0Q_CS~\x95\x8b\xfd\xf8\xd4\xeb$\t\xff\x90\x9c2\xdb\x13\xef\xab\xb9\x83\x00iKO\xc5\x87\xee@\xe4\x0b\x97\xfa13\x12\x12\xa9\xe5\ti\xb3\x03J+\x8fP\x9e\xbd\x12'

In [6]:
# ARX结构，addition：2个32bit数据，先变成整数，在Z2^32上，R：循环移位，X：异或，没有用有限域S盒线性变换等
# ChaCha算法是基于ARX结构的
# TLS1.4种系列密码基本都是ChaCha，分组密码是AES
# 序列密码没有分组模式，输入是IV和种子密钥key，输出密钥流，要保证输出密钥流周期比较长
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


plaintext = b"this is a paintext, this is a paintext~"
key = key = os.urandom(32)
nonce = os.urandom(16)

algorithm = algorithms.ChaCha20(key, nonce)
# ChaCha20 是一个流密码，mode 必须为 None
cipher = Cipher(algorithm, mode=None)

# 1. 加密
encryptor = cipher.encryptor()
ct = encryptor.update(plaintext)

# 2. 解密
decryptor = cipher.decryptor()
decryptor.update(ct)

b'this is a paintext, this is a paintext~'

In [5]:
# 调用pycryptodemo实现公钥密码
# RSA算法：p,q有时候可以作为私钥，有时候可以不作为，可以直接销毁
# 怎样生成两个大素数：素性判定（给定大素数判断是不是素数，目前用的比较多的还是概率性的算法，比如基于椭圆曲线的）
# 概率性算法：蒙特卡洛：一定能给结果，但不一定是对的，拉斯维加斯：如果算不出来，就什么都不输出
# 两个素数不能离得太近，例如孪生素数，可以通过n=p(p+60)，通过求一个二次方程就能找出来了
# p-1/2和q-1/2也不能离得太近
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

key = RSA.generate(2048)
# 用的公钥300多个整数位/2048个比特位，现在1024逐渐弃用，512可以用服务器一两天的时间攻击
# 使用export_key()生成私钥
private_key = key.export_key()
public_key = key.publickey().export_key()

data = b"123456"
# 把刚才生成好的公钥导进来，作为加密要用的公钥
pk = RSA.import_key(public_key)
# 得到了cipher,算法+工作模式，有了cipher才能去加密
# OPEA是把数据处理成RSA能处理的数据格式
cipher = PKCS1_OAEP.new(pk)
ct = cipher.encrypt(data)
print(ct)
# 解密，需要用到私钥
# 首先需要把私钥导入进来
privk = RSA.import_key(private_key)
cipher = PKCS1_OAEP.new(privk)
data = cipher.decrypt(ct)
print(data)


b'!E\x12\xd5\x06:(\xaa\x12\xc9H\t\xd6\x90J\x82\x9e\xa3\xd0\x1b%\xb0\xba\x8b\xc4">\xe0B\xb2\xfa\xa0\t%\xaag\xca\x8f\xa4|\x8c\x95.\x91K\xbf\x95\xb57\xfd\xa2&3\xb5\x8e\xf25\xbc\xa9\xf0qZ\x07D&\xce\xc2=\x95m:\xc7\xd0 o\x1a$T\xc0\xc3\xbc\xae\x9eQ\x02\x86\x11\xa6\xb4\xfa\xf4\x9eL=\xbfh\xd8\x02!\xb1L\xd0\xf7`\xfb1\xe5\r\x95$\xdd\xeb\xee{\xb5\xecS\xec7\xb0\x89\xdaFzOh\xaf]C\x18K(A\x0b\xa6nqM\x006\xdc\xc9:\xcb\t#|\xad\xf7\x91\xb2"\xc9A\x947\xdaq\xa7\x90\xe7\xdd\xf9\x1f\xb8\xf0\xba"\xaa\xd5m\xfd\xc6\xb7\xf8\xf4\x7f\xce\x1fn7q\xf7uS\x9b\x82\x89\xe5\xef\xd4T8\xab\xea\x1c\xa4\x8b\xaa\x04\xf22\xe5\xa2\xd5h\xf3k\x07\xd4\xa8\xa8\x0f/\xe7\xa7b(\xe7\xa9\xc6`m\xb3cyR<\xda\x10\x1ci\tbk0Hi.6\x15"F\xdd\xe3\xb9\xb0D\xd3\x8e\xe7\xb0\x1c\x05\x1dc'
b'123456'
